# Library Things

In [1]:
!pip install duckdb --upgrade

In [2]:
!pip install polars --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 10.0 MB/s eta 0:00:00
  Attempting uninstall: polars
    Found existing installation: polars 1.7.1
    Uninstalling polars-1.7.1:
      Successfully uninstalled polars-1.7.1


In [3]:
import duckdb
import polars as pl
from IPython.display import HTML

In [4]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


# The Tables Preparation

In [5]:
# Connect to DuckDB
conn = duckdb.connect()

# Load spatial extension for Excel files
conn.sql('INSTALL spatial;')
conn.sql('LOAD spatial;')

# Create the temporary table for storing financial raw data
create_financial_raw_table_query = """
CREATE TEMPORARY TABLE financial_raw (
    segment VARCHAR,
    country VARCHAR,
    product VARCHAR,
    discount_band VARCHAR,
    unit_sold DOUBLE,
    manufacturing DOUBLE,
    sale_price DOUBLE,
    gross_sales DOUBLE,
    discounts DOUBLE,
    sales DOUBLE,
    cogs DOUBLE,
    profit DOUBLE,
    date DATE,
    month_number INTEGER,
    month_name VARCHAR,
    year INTEGER
);
"""

# Execute the query to create the temporary table
conn.execute(create_financial_raw_table_query)

In [6]:
# Define path for the CSV files
financial_raw_csv_file_path = "/content/drive/MyDrive/Nawatech Technical Test/Financial Sample.xlsx"

# Load the data from the CSV file
load_financial_raw_table_query = f"""
                                      INSERT INTO financial_raw
                                      SELECT * FROM st_read('{financial_raw_csv_file_path}', layer='Sheet1', open_options=['HEADERS=FORCE', 'FIELD_TYPES=AUTO']);
                                  """

# Execute the query to load the data from the CSV file
conn.execute(load_financial_raw_table_query)

In [7]:
# Verify the insertion by selecting some data
conn.sql("SELECT * FROM financial_raw LIMIT 5;").show()

┌────────────┬─────────┬───────────┬───────────────┬───────────┬───────────────┬────────────┬─────────────┬───────────┬─────────┬─────────┬─────────┬────────────┬──────────────┬────────────┬───────┐
│  segment   │ country │  product  │ discount_band │ unit_sold │ manufacturing │ sale_price │ gross_sales │ discounts │  sales  │  cogs   │ profit  │    date    │ month_number │ month_name │ year  │
│  varchar   │ varchar │  varchar  │    varchar    │  double   │    double     │   double   │   double    │  double   │ double  │ double  │ double  │    date    │    int32     │  varchar   │ int32 │
├────────────┼─────────┼───────────┼───────────────┼───────────┼───────────────┼────────────┼─────────────┼───────────┼─────────┼─────────┼─────────┼────────────┼──────────────┼────────────┼───────┤
│ Government │ Canada  │ Carretera │ None          │    1618.5 │           3.0 │       20.0 │     32370.0 │       0.0 │ 32370.0 │ 16185.0 │ 16185.0 │ 2014-01-01 │            1 │ January    │  2014 │
│ Gov

# Perform the Data Transformation

**Basic Data Inspection: Summary Statistics**

In [8]:
financial_raw_df = conn.sql("SELECT * FROM financial_raw;").pl()

In [9]:
# Get summary statistics for string and numerical columns
summary = financial_raw_df.describe()

html_output = summary._repr_html_()
display(HTML(html_output))

statistic,segment,country,product,discount_band,unit_sold,manufacturing,sale_price,gross_sales,discounts,sales,cogs,profit,date,month_number,month_name,year
str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,str,f64
"""count""","""700""","""700""","""700""","""700""",700.0,700.0,700.0,700.0,700.0,700.0,700.0,700.0,"""700""",700.0,"""700""",700.0
"""null_count""","""0""","""0""","""0""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""0""",0.0,"""0""",0.0
"""mean""",null,null,null,null,1608.294286,96.477143,118.428571,182759.426429,13150.354629,169609.0718,145475.211429,24133.860371,"""2014-04-28 21:36:00""",7.9,null,2013.75
"""std""",null,null,null,null,867.427859,108.602612,136.775515,254262.284378,22962.928775,236726.34691,203865.506118,42760.626563,null,3.377321,null,0.433322
"""min""","""Channel Partners""","""Canada""","""Amarilla""","""High""",200.0,3.0,7.0,1799.0,0.0,1655.08,918.0,-40617.5,"""2013-09-01""",1.0,"""April""",2013.0
"""25%""",null,null,null,null,905.0,5.0,12.0,17409.0,801.0,15928.0,7490.0,2807.2,"""2014-01-01""",6.0,null,2014.0
"""50%""",null,null,null,null,1545.0,10.0,20.0,37980.0,2596.5,35585.6,22550.0,9242.6,"""2014-06-01""",9.0,null,2014.0
"""75%""",null,null,null,null,2227.5,250.0,300.0,277200.0,15913.125,260580.0,245310.0,22662.0,"""2014-09-01""",10.0,null,2014.0
"""max""","""Small Business""","""United States of America""","""Velo""","""None""",4492.5,260.0,350.0,1.2075e6,149677.5,1.1592e6,950625.0,262200.0,"""2014-12-01""",12.0,"""September""",2014.0


**Check for Missing Values**

In [10]:
# Count of null values per column
null_counts = financial_raw_df.null_count()

html_output = null_counts._repr_html_()
display(HTML(html_output))

segment,country,product,discount_band,unit_sold,manufacturing,sale_price,gross_sales,discounts,sales,cogs,profit,date,month_number,month_name,year
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Check the "date" timeframe (Sept 2013 - Dec 2023)**

In [11]:
# Create a new column to check if date is between September 2013 and December 2014
temp_df = financial_raw_df.with_columns(
    pl.col("date")
    .is_between(pl.date(2013, 9, 1), pl.date(2014, 12, 31))
    .alias("is_date_correct")
)

In [12]:
# Filter rows where value is False and check the row count
out_of_timeframe_count = temp_df.filter(pl.col("is_date_correct") == False).height

print(f"The row count for out of timeframe column: {out_of_timeframe_count}")

The row count for out of timeframe column: 0


**Check the "month_number" and "month_name" (1-12, Jan-Dec)**

In [13]:
# Create a new column to check if month_number is between 1 and 12
temp_df = temp_df.with_columns(
    pl.col("month_number")
    .is_between(1, 12)
    .alias("is_month_number_correct")
)

# Create a new column to check if year is between 2013 and 2014
temp_df = temp_df.with_columns(
    pl.col("year")
    .is_between(2013, 2014)
    .alias("is_year_correct")
)

# List of valid months
valid_months = ["January", "February", "March", "April", "May", "June",
                "July", "August", "September", "October", "November", "December"]
# Create a new column to check if month_name is between January and December
temp_df = temp_df.with_columns(
    pl.col("month_name")
    .is_in(valid_months)
    .alias("is_month_name_correct")
)

In [14]:
# Filter rows where 'is_date_correct' is False and check the row count
incorrect_month_number_count = temp_df.filter(pl.col("is_month_number_correct") == False).height
incorrect_year_count = temp_df.filter(pl.col("is_year_correct") == False).height
incorrect_month_name_count = temp_df.filter(pl.col("is_month_name_correct") == False).height

print(f"The row count for false month_number column: {incorrect_month_number_count}")
print(f"The row count for false year column: {incorrect_year_count}")
print(f"The row count for false month_name column: {incorrect_month_name_count}")

The row count for false month_number column: 0
The row count for false year column: 0
The row count for false month_name column: 0


**Check the Calculation of "Gross Sales", "Sales", "COGS", and "Profit" columns**

In [15]:
# Calculate the expected values based on the schema
temp_df = temp_df.with_columns([
    (pl.col("unit_sold") * pl.col("sale_price")).alias("calculated_gross_sales"),
    (pl.col("gross_sales") - pl.col("discounts")).alias("calculated_sales"),
    (pl.col("unit_sold") * pl.col("manufacturing")).alias("calculated_cogs"),
    (pl.col("sales") - pl.col("cogs")).alias("calculated_profit")
])

temp_df = temp_df.with_columns([
    (pl.col("gross_sales") == pl.col("calculated_gross_sales")).alias("is_correct_gross_sales"),
    (pl.col("sales") == pl.col("calculated_sales")).alias("is_correct_sales"),
    (pl.col("cogs") == pl.col("calculated_cogs")).alias("is_correct_cogs"),
    (pl.col("profit") == pl.col("calculated_profit")).alias("is_correct_profit")
])

# Compare with the table values
comparison = temp_df.select([
    "gross_sales", "calculated_gross_sales",
    "sales", "calculated_sales",
    "cogs", "calculated_cogs",
    "profit", "calculated_profit",
    "is_correct_gross_sales", "is_correct_sales",
    "is_correct_cogs", "is_correct_profit"
])

print(comparison)

shape: (700, 12)
┌───────────┬───────────┬──────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ gross_sal ┆ calculate ┆ sales    ┆ calculate ┆ … ┆ is_correc ┆ is_correc ┆ is_correc ┆ is_correc │
│ es        ┆ d_gross_s ┆ ---      ┆ d_sales   ┆   ┆ t_gross_s ┆ t_sales   ┆ t_cogs    ┆ t_profit  │
│ ---       ┆ ales      ┆ f64      ┆ ---       ┆   ┆ ales      ┆ ---       ┆ ---       ┆ ---       │
│ f64       ┆ ---       ┆          ┆ f64       ┆   ┆ ---       ┆ bool      ┆ bool      ┆ bool      │
│           ┆ f64       ┆          ┆           ┆   ┆ bool      ┆           ┆           ┆           │
╞═══════════╪═══════════╪══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 32370.0   ┆ 32370.0   ┆ 32370.0  ┆ 32370.0   ┆ … ┆ true      ┆ true      ┆ false     ┆ true      │
│ 26420.0   ┆ 26420.0   ┆ 26420.0  ┆ 26420.0   ┆ … ┆ true      ┆ true      ┆ false     ┆ true      │
│ 32670.0   ┆ 32670.0   ┆ 32670.0  ┆ 32670.0   ┆ … ┆ true      ┆ true     

In [16]:
# Filter rows where value is False and check the row count
false_gross_sales_count = temp_df.filter(pl.col("is_correct_gross_sales") == False).height
false_sales_count = temp_df.filter(pl.col("is_correct_sales") == False).height
false_cogs_count = temp_df.filter(pl.col("is_correct_cogs") == False).height
false_profit_count = temp_df.filter(pl.col("is_correct_profit") == False).height

print(f"The row count for false gross sales column: {false_gross_sales_count}")
print(f"The row count for false sales column: {false_sales_count}")
print(f"The row count for false cogs column: {false_cogs_count}")
print(f"The row count for false profit column: {false_profit_count}")

The row count for false gross sales column: 0
The row count for false sales column: 12
The row count for false cogs column: 536
The row count for false profit column: 15


**Check other column values**

In [17]:
# Get unique Segment
unique_segment = temp_df.select(pl.col("segment").unique())

unique_segment

segment
str
"""Government"""
"""Midmarket"""
"""Small Business"""
"""Enterprise"""
"""Channel Partners"""


In [18]:
# Get unique Country
unique_country = temp_df.select(pl.col("country").unique())

unique_country

country
str
"""Canada"""
"""France"""
"""Mexico"""
"""United States of America"""
"""Germany"""


In [19]:
# Get unique Product
unique_product = temp_df.select(pl.col("product").unique())

unique_product

product
str
"""Carretera"""
"""Amarilla"""
"""Paseo"""
"""Montana"""
"""VTT"""
"""Velo"""


In [20]:
# Group by 'Discount Band' and calculate min and max for 'Discounts'
unique_discount_band = temp_df.group_by("discount_band").agg([
    pl.col("discounts").min().alias("min_discounts"),
    pl.col("discounts").max().alias("max_discounts")
])

unique_discount_band

discount_band,min_discounts,max_discounts
str,f64,f64
"""None""",0.0,0.0
"""Low""",18.41,48300.0
"""High""",274.4,149677.5
"""Medium""",110.46,102667.5


**Data Transformation**

In [21]:
# Apply the correct calculation to the 'gross_sales' column where 'is_correct_gross_sales' is False
temp_df = temp_df.with_columns(
    pl.when(pl.col("is_correct_gross_sales") == False)
    .then(pl.col("unit_sold") * pl.col("sale_price"))
    .otherwise(pl.col("gross_sales"))
    .alias("correct_gross_sales")
)

# Apply the correct calculation to the 'sales' column where 'is_correct_sales' is False
temp_df = temp_df.with_columns(
    pl.when(pl.col("is_correct_sales") == False)
    .then(pl.col("gross_sales") - pl.col("discounts"))
    .otherwise(pl.col("sales"))
    .alias("correct_sales")
)

# Apply the correct calculation to the 'cogs' column where 'is_correct_cogs' is False
temp_df = temp_df.with_columns(
    pl.when(pl.col("is_correct_cogs") == False)
    .then(pl.col("unit_sold") * pl.col("manufacturing"))
    .otherwise(pl.col("cogs"))
    .alias("correct_cogs")
)

# Apply the correct calculation to the 'profit' column where 'is_correct_profit' is False
temp_df = temp_df.with_columns(
    pl.when(pl.col("is_correct_profit") == False)
    .then(pl.col("sales") - pl.col("cogs"))
    .otherwise(pl.col("profit"))
    .alias("correct_profit")
)

In [22]:
# Drop and rename some columns
columns_to_be_dropped = [
    "is_date_correct",
    "is_month_number_correct",
    "is_year_correct",
    "is_month_name_correct",
    "calculated_gross_sales",
    "calculated_sales",
    "calculated_cogs",
    "calculated_profit",
    "is_correct_gross_sales",
    "is_correct_sales",
    "is_correct_cogs",
    "is_correct_profit",
    "gross_sales",
    "sales",
    "cogs",
    "profit"
]
columns_to_be_renamed = {
    "correct_gross_sales": "gross_sales",
    "correct_sales": "sales",
    "correct_cogs": "cogs",
    "correct_profit": "profit"
}

financial_raw_df = temp_df.drop(columns_to_be_dropped).rename(columns_to_be_renamed)

In [23]:
financial_raw_df

segment,country,product,discount_band,unit_sold,manufacturing,sale_price,discounts,date,month_number,month_name,year,gross_sales,sales,cogs,profit
str,str,str,str,f64,f64,f64,f64,date,i32,str,i32,f64,f64,f64,f64
"""Government""","""Canada""","""Carretera""","""None""",1618.5,3.0,20.0,0.0,2014-01-01,1,"""January""",2014,32370.0,32370.0,4855.5,16185.0
"""Government""","""Germany""","""Carretera""","""None""",1321.0,3.0,20.0,0.0,2014-01-01,1,"""January""",2014,26420.0,26420.0,3963.0,13210.0
"""Midmarket""","""France""","""Carretera""","""None""",2178.0,3.0,15.0,0.0,2014-06-01,6,"""June""",2014,32670.0,32670.0,6534.0,10890.0
"""Midmarket""","""Germany""","""Carretera""","""None""",888.0,3.0,15.0,0.0,2014-06-01,6,"""June""",2014,13320.0,13320.0,2664.0,4440.0
"""Midmarket""","""Mexico""","""Carretera""","""None""",2470.0,3.0,15.0,0.0,2014-06-01,6,"""June""",2014,37050.0,37050.0,7410.0,12350.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Small Business""","""France""","""Amarilla""","""High""",2475.0,260.0,300.0,111375.0,2014-03-01,3,"""March""",2014,742500.0,631125.0,643500.0,12375.0
"""Small Business""","""Mexico""","""Amarilla""","""High""",546.0,260.0,300.0,24570.0,2014-10-01,10,"""October""",2014,163800.0,139230.0,141960.0,2730.0
"""Government""","""Mexico""","""Montana""","""High""",1368.0,5.0,7.0,1436.4,2014-02-01,2,"""February""",2014,9576.0,8139.6,6840.0,1299.6


**Save the transformed data**

In [24]:
# Save the DataFrame to a CSV file
financial_raw_df.write_csv('/content/drive/MyDrive/Nawatech Technical Test/transformed_financial_raw.csv')
print("Data has been saved to '/content/drive/MyDrive/Nawatech Technical Test/transformed_financial_raw.csv'")

Data has been saved to '/content/drive/MyDrive/Nawatech Technical Test/transformed_financial_raw.csv'


# Perform Normalization

In [41]:
# 1. Create the Segment table
create_segment_raw_table_query = """
DROP SEQUENCE IF EXISTS segment_id_sequence CASCADE;
CREATE SEQUENCE segment_id_sequence START 1;
CREATE TABLE segment (
    segment_id INTEGER DEFAULT nextval('memory.segment_id_sequence'),
    segment_name VARCHAR,
    PRIMARY KEY (segment_id)
);
"""
insert_segment_raw_table_query = """
INSERT INTO segment BY NAME (SELECT DISTINCT segment AS segment_name FROM financial_raw);
"""

# 2. Create the Country table
create_country_table_query = """
DROP SEQUENCE IF EXISTS country_id_sequence CASCADE;
CREATE SEQUENCE country_id_sequence START 1;
CREATE TABLE country (
    country_id INTEGER DEFAULT nextval('memory.country_id_sequence'),
    country_name VARCHAR,
    PRIMARY KEY (country_id)
);
"""
insert_country_table_query = """
INSERT INTO country BY NAME (SELECT DISTINCT country AS country_name FROM financial_raw);
"""

# 3. Create the Product table
create_product_table_query = """
DROP SEQUENCE IF EXISTS product_id_sequence CASCADE;
CREATE SEQUENCE product_id_sequence START 1;
CREATE TABLE product (
    product_id INTEGER DEFAULT nextval('memory.product_id_sequence'),
    product_name VARCHAR,
    PRIMARY KEY (product_id)
);
"""
insert_product_table_query = """
INSERT INTO product BY NAME (SELECT DISTINCT product AS product_name FROM financial_raw);
"""

# 4. Create the Discount Band table
create_discount_band_raw_table_query = """
DROP SEQUENCE IF EXISTS discount_band_id_sequence CASCADE;
CREATE SEQUENCE discount_band_id_sequence START 1;
CREATE TABLE discount_band (
    discount_band_id INTEGER DEFAULT nextval('memory.discount_band_id_sequence'),
    discount_band_name VARCHAR,
    PRIMARY KEY (discount_band_id)
);
"""
insert_discount_band_table_query = """
INSERT INTO discount_band BY NAME (SELECT DISTINCT discount_band AS discount_band_name FROM financial_raw);
"""

# 5. Create the Transaction table with foreign keys
create_transaction_raw_table_query = """
DROP SEQUENCE IF EXISTS transaction_id_sequence CASCADE;
CREATE SEQUENCE transaction_id_sequence START 1;
CREATE TABLE transaction (
    transaction_id INTEGER DEFAULT nextval('memory.transaction_id_sequence'),
    segment_id INTEGER,
    country_id INTEGER,
    product_id INTEGER,
    discount_band_id INTEGER,
    unit_sold DOUBLE,
    manufacturing_price DOUBLE,
    sale_price DOUBLE,
    gross_sales DOUBLE,
    discounts DOUBLE,
    sales DOUBLE,
    cogs DOUBLE,
    profit DOUBLE,
    sale_date DATE,
    PRIMARY KEY (transaction_id),
    FOREIGN KEY (segment_id) REFERENCES segment(segment_id),
    FOREIGN KEY (country_id) REFERENCES country(country_id),
    FOREIGN KEY (product_id) REFERENCES product(product_id),
    FOREIGN KEY (discount_band_id) REFERENCES discount_band(discount_band_id)
);
"""
insert_transaction_raw_table_query = """
INSERT INTO transaction (
    segment_id, country_id, product_id, discount_band_id, unit_sold, manufacturing_price,
    sale_price, gross_sales, discounts, sales, cogs, profit, sale_date
)
SELECT
    s.segment_id AS segment_id,
    c.country_id AS country_id,
    p.product_id AS product_id,
    db.discount_band_id AS discount_band_id,
    fr.unit_sold AS unit_sold,
    fr.manufacturing AS manufacturing,
    fr.sale_price AS sale_price,
    fr.gross_sales AS gross_sales,
    fr.discounts AS discounts,
    fr.sales AS sales,
    fr.cogs AS cogs,
    fr.profit AS profit,
    fr.date AS sale_date
FROM financial_raw AS fr
JOIN segment s
ON fr.segment = s.segment_name
JOIN country c
ON fr.country = c.country_name
JOIN product p
ON fr.product = p.product_name
JOIN discount_band db
ON fr.discount_band = db.discount_band_name;
"""

# Execute the queries to create the tables and insert data
conn.execute(create_segment_raw_table_query)
conn.execute(insert_segment_raw_table_query)

conn.execute(create_country_table_query)
conn.execute(insert_country_table_query)

conn.execute(create_product_table_query)
conn.execute(insert_product_table_query)

conn.execute(create_discount_band_raw_table_query)
conn.execute(insert_discount_band_table_query)

conn.execute(create_transaction_raw_table_query)
conn.execute(insert_transaction_raw_table_query)

In [42]:
# Verify the insertion by selecting some data
conn.sql("SELECT * FROM transaction LIMIT 5;").show()

┌────────────────┬────────────┬────────────┬────────────┬──────────────────┬───────────┬─────────────────────┬────────────┬─────────────┬───────────┬─────────┬─────────┬─────────┬────────────┐
│ transaction_id │ segment_id │ country_id │ product_id │ discount_band_id │ unit_sold │ manufacturing_price │ sale_price │ gross_sales │ discounts │  sales  │  cogs   │ profit  │ sale_date  │
│     int32      │   int32    │   int32    │   int32    │      int32       │  double   │       double        │   double   │   double    │  double   │ double  │ double  │ double  │    date    │
├────────────────┼────────────┼────────────┼────────────┼──────────────────┼───────────┼─────────────────────┼────────────┼─────────────┼───────────┼─────────┼─────────┼─────────┼────────────┤
│              1 │          1 │          3 │          1 │                2 │    1618.5 │                 3.0 │       20.0 │     32370.0 │       0.0 │ 32370.0 │ 16185.0 │ 16185.0 │ 2014-01-01 │
│              2 │          1 │    

In [43]:
# Based on normalized tables, create a SQL query for finding TOP 5 product sold in Mexico in 2013.
query1 = """
SELECT p.product_name, SUM(t.unit_sold) AS total_units_sold
FROM transaction t
JOIN country c ON t.country_id = c.country_id
JOIN product p ON t.product_id = p.product_id
WHERE c.country_name = 'Mexico'
AND EXTRACT(YEAR FROM t.sale_date) = 2013
GROUP BY p.product_name
ORDER BY total_units_sold DESC
LIMIT 5;
"""
conn.sql(query1).show()

┌──────────────┬──────────────────┐
│ product_name │ total_units_sold │
│   varchar    │      double      │
├──────────────┼──────────────────┤
│ Paseo        │          15529.0 │
│ Carretera    │           8222.0 │
│ Montana      │           6922.0 │
│ Velo         │           6293.0 │
│ Amarilla     │           6263.0 │
└──────────────┴──────────────────┘



In [45]:
# Based on normalized tables, create a SQL query to compare the profit of each product in 2013 and 2014.
query2 = """
SELECT
    p.product_name,
    ROUND(SUM(CASE WHEN EXTRACT(YEAR FROM t.sale_date) = 2013 THEN t.profit ELSE 0 END), 2) AS profit_2013,
    ROUND(SUM(CASE WHEN EXTRACT(YEAR FROM t.sale_date) = 2014 THEN t.profit ELSE 0 END), 2) AS profit_2014
FROM transaction t
JOIN product p ON t.product_id = p.product_id
GROUP BY p.product_name
ORDER BY p.product_name;
"""
conn.sql(query2).show()

┌──────────────┬─────────────┬─────────────┐
│ product_name │ profit_2013 │ profit_2014 │
│   varchar    │   double    │   double    │
├──────────────┼─────────────┼─────────────┤
│ Amarilla     │   781949.53 │  2032154.53 │
│ Carretera    │    38768.86 │  1788036.03 │
│ Montana      │   457758.04 │  1656996.84 │
│ Paseo        │  1099853.09 │  3697584.86 │
│ VTT          │   878185.23 │  2156422.79 │
│ Velo         │   621949.76 │   1684042.7 │
└──────────────┴─────────────┴─────────────┘

